# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


импорт библиотек

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [3]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [4]:
oll_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/uslugi/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',
         
]

In [5]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'url', 'date', 'availability', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for j in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'url' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'date' : pd.to_datetime('today').normalize(),
                             'availability' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

179  -  16  -  23  -  100  -  33  -  60  -  10  -  4  -  31  -  25  -  5  -  5  -  81 

In [6]:
dft = parsing_2(oll_url)
dft

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

,title,url,date,availability,id,price,old_price
0,"Стойка коммутацион.Netrack (NP5104) 19', 4U 482*177*530mm, black",https://tiraet.com/catalog/Komplektujuschie/komplektujuschie/Korpusa/stoyka_kommutatsion_netrack_np5104_19_4u_482_177_530mm_black/,2024-07-03,Достаточно,87094,NaN,NaN
1,Переходник для мыши COM в PS/2 (A-235),https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/Perehodnik-dlja-myshi-COM-v-PS2-A-235/,2024-07-03,Много,284,5.0,NaN
2,"Кабель MINI USB2.0 (A - MINI USB 4PM)1,8м (CC-USB2-AM4P-6)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/kabel-mini-usb20-a---mini-usb-4pm18m-cc-usb2-am4p-6/,2024-07-03,Много,8683,5.0,NaN
3,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм, PVC (49979) Black 1шт",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/box_dlya_cd_dvd_verbatim_slim_na_1_disk_120mm_pvc_49979_black_1sht/,2024-07-03,Много,85078,6.6,NaN
4,"Диск CD-R 700MB VERBATIM, 52X (1шт) EXTRA PROTECTION(43411/43351/43437)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_cdr_700mb_verbatim_52x_1sht_extra_protection_43411_43351_43437/,2024-07-03,Много,12819,6.6,NaN
...,...,...,...,...,...,...,...
11637,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka-dlja-plotterov-canon-pf-03-2251b001/,2024-07-03,Мало,27477,7920.0,NaN
11638,Узел термозакрепления (для сетевых апп)CANON iR2202/2204(FM1-F162-0XX),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/uzel_termozakrepleniya_dlya_setevykh_app_canon_ir2202_2204_fm1_f162_0xx/,2024-07-03,Мало,55309,8085.0,NaN
11639,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka_dlya_plotterov_canon_pf_04_3630b001/,2024-07-03,Мало,56144,8250.0,NaN
11640,Картридж EPSON MX-80/LX-300/400/800/850/880 Patron,https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-matrichnym-printeram-kartridzhi-lenty/kartridji_matr/kartridzh-epson-lx-300400800850880mx-80/,2024-07-03,Нет в наличии,249,38.0,NaN


dft.to_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

dft = pd.read_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [7]:
tiraet_group = pd.read_csv('csv/atiraet_group.csv')

In [8]:
tiraet_group = tiraet_group.drop_duplicates(keep= False )

In [9]:
dft = dft.merge(tiraet_group, how='left', on='id')

In [10]:
t_df_new = dft[(dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna())]

In [11]:
dft = dft[~((dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna()))]

In [12]:
def find_groups(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_1 = soup.findAll('a', class_='breadcrumbs__link colored_theme_hover_bg-el-svg')
    elem_2 = soup.findAll('div', class_='breadcrumbs__item')
    try:
        if len(elem_1) <=2:
            if len(elem_2) <= 4:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            else:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_2[4].text.replace(',','').replace(' ','_').replace('-','_').lower())
        else:
            d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_16 = 'T_'+(elem_1[2].text.replace(',','').replace(' ','_').replace('-','_').lower())
    except:
        d_14 = 'err_'
        d_15 = 'err_'
        d_16 = 'err_'
        
    return pd.Series([ (d_14), (d_15), (d_16)])

In [13]:
tqdm.pandas() 
t_df_new[['_category','_subcategory','_group']] = t_df_new['url'].progress_apply(find_groups)

  0%|          | 0/13 [00:00<?, ?it/s]

In [14]:
t_df_new._category.unique()

array(['T_компьютерная_техника', 'T_для_дома', 'err_',
       'T_спорт_и_увлечения'], dtype=object)

In [15]:
t_df_new._subcategory.unique()

array(['T_комплектующие', 'T_аксессуары', 'T_мебель', 'err_',
       'T_персональный_транспорт'], dtype=object)

In [16]:
t_df_new._group.unique()

array(['T_приводы_оптические', 'T_диски_дискеты', 'T_кресла', 'err_',
       'T_электросамокат'], dtype=object)

In [17]:
t_data = t_df_new[~(t_df_new['_category']=='err_')].reset_index(drop=True)
t_data = t_data[['id', '_category','_subcategory','_group' ]]
t_data = t_data.sort_values(['_category','_subcategory', '_group'])

In [18]:
tiraet_group = pd.concat([tiraet_group, t_data], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)
tiraet_group.to_csv('csv/atiraet_group.csv', index=False)  

In [19]:
dft = pd.concat([dft, t_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)

dft.to_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [20]:
dft[dft['_category']=='err_'].id.count()

1

In [21]:
dft[dft['_category']=='err_']

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group
11641,"Музыкальный центр PANASONIC SC-AKX320GSK (2.0, 450Вт, CD, FM, USB, SD, BT, Д/У) BLACK",https://tiraet.com/catalog/bytovaya/melkaja-bytovaja-tehnika/muzykalnye-centry/muzykalnyy_tsentr_panasonic_sc_akx320gsk_2_0_450vt_cd_fm_usb_sd_bt_d_u_black/,2024-07-03,Мало,87154,3799.0,NaN,err_,err_,err_


In [22]:
dft['error_group'] = 0
dft.loc[dft['_category']=='err', 'error_group'] = 1

In [23]:
dft.head(2)

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group,error_group
0,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,2024-07-03,Мало,78604,660.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0
1,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,2024-07-03,Мало,78235,1419.0,NaN,T_автотовары,T_автомагнитола,T_автомагнитола,0


In [24]:
len(t_data)

12

In [25]:
for i in dft['_category'].unique():
    t = dft[dft['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

T_автотовары
['T_автомагнитола' 'T_автопылесос' 'T_авторегистратор' 'T_автохолодильник'
 'T_динамики' 'T_компрессоры' 'T_крепления_смартфонов/планшетов'
 'T_модуляторы_fm' 'T_навигаторы_gps']
----------------------------------------------------------------------------------------------------
T_безопасность
['T_видеонаблюдение' 'T_домофоны' 'T_монтаж_и_питание'
 'T_оборудование_poe' 'T_сигнализация_охранно_пожарная' 'T_скуд'
 'T_умный_дом']
----------------------------------------------------------------------------------------------------
T_бытовая_техника
['T_гигиена_здоровье_уход' 'T_климатическая_техника'
 'T_крупная_бытовая_техника' 'T_мелкая_бытовая_техника' 'T_телевизоры'
 'T_техника_для_кухни' 'T_часы_наручные']
----------------------------------------------------------------------------------------------------
T_для_дома
['T_бытовая_химия' 'T_запасные_части' 'T_инструменты' 'T_крепления'
 'T_кухонная_утварь' 'T_мебель' 'T_текстиль' 'T_хозяйственный_инвентарь']
-----------------

# Аналитика акционных товаров

In [26]:
brend = pd.read_excel('csv/brend.xlsx')

In [27]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [28]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
dft['brend'] = dft['title'].progress_apply(brends)

  0%|          | 0/11642 [00:00<?, ?it/s]

создаем столбец с брендами

In [29]:
dft['action_sale'] = 0
dft.loc[dft['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [30]:
dft['sale'] = (100 - ((dft['price']*100)/dft['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [31]:
def count_group(x):
    return dft[dft['_group']==x]['id'].count()    
tqdm.pandas()
dft['count_group'] = dft['_group'].progress_apply(count_group)

  0%|          | 0/11642 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [32]:
def count_group_presence(x):
    return dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['id'].count()   
tqdm.pandas()
dft['activ_count_group'] = dft['_group'].progress_apply(count_group_presence)

  0%|          | 0/11642 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [33]:
def dif_price(x):    
    max_price = dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['price'].max()
    min_price = dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
dft['dif_price'] = dft['_group'].progress_apply(dif_price)

  0%|          | 0/11642 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [34]:
dft['group_prise'] = dft.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [35]:
dft = dft[['url', 'id', '_category', '_subcategory', '_group', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group', 'error_group']]

In [36]:
dft['tir'] = 1

In [37]:
dft = dft.drop_duplicates().reset_index(drop=True)

In [38]:
preprocessing(dft)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,78604,T_автотовары,T_автомагнитола,T_автомагнитола,sencor,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",2024-07-03,660.0,NaN,0,NaN,бюджет,3.331818,Мало,6,6,0,1
1,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,78235,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",2024-07-03,1419.0,NaN,0,NaN,стандарт,3.331818,Мало,6,6,0,1
2,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kmm_bt209_fm_am_bt_usb_aux_rca_4_50w_iso_1din/,80185,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KMM-BT209 (FM/AM, BT, USB, AUX, RCA, 4*50W, ISO, 1DIN)",2024-07-03,1735.0,NaN,0,NaN,дорогой,3.331818,Достаточно,6,6,0,1
3,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_jvc_kdx_382bt_fm_am_bluetooth_usb_4_50w_iso_aux_1din/,80184,T_автотовары,T_автомагнитола,T_автомагнитола,jvc,"Автомагнитола JVC KDX-382BT (FM/AM, Bluetooth, USB, 4*50W, ISO, AUX, 1DIN)",2024-07-03,1815.0,NaN,0,NaN,элитный,3.331818,Мало,6,6,0,1
4,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_jvc_kdt_702bt_fm_am_bluetooth_cd_usb_4_50w_1din/,80183,T_автотовары,T_автомагнитола,T_автомагнитола,jvc,"Автомагнитола JVC KDT-702BT (FM/AM, Bluetooth, CD, USB, 4*50W, 1DIN)",2024-07-03,2099.0,NaN,0,NaN,элитный,3.331818,Достаточно,6,6,0,1


****************************************************************************************************
таблица имеет 11642 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    2
old_price            10119
action_sale              0
sale                 10119
group_prise              2
dif_price                2
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            86.9
action_sale           0.0
sale                 86.9
group_prise           0.0
dif_price             0.0
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11642 entries, 0 to 11641
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                11642 non-null  object        
 1   id                 11642 non-null  object        
 2   _category          11642 non-null  object        
 3   _subcategory       11642 non-null  object        
 4   _group             11642 non-null  object        
 5   brend              11642 non-null  object        
 6   title              11642 non-null  object        
 7   date               11642 non-null  datetime64[ns]
 8   price              11640 non-null  float64   

None

# Парсинг Hi

In [39]:
def h_find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [40]:
def h_find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [41]:
def h_find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

функция создающая столбец с названием товара

In [42]:
def h_find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

функция создающая столбец с продажной ценой товара

In [43]:
def h_find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

функция созжающая столбец с предидущей ценой для акционного товара

In [44]:
def h_find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

Функция создающая столбец с урл адресом товара

In [45]:
def h_parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        attempt = 1
        while q>0 or attempt<4:             
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
                attempt = 1                
            else:
                if attempt == 3:                    
                    q = 0
                    attempt += 1
                else:
                    attempt += 1                    
    data['id'] = data['url'].apply(h_find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(h_find_availability)
    data['title'] = data['url'].apply(h_find_title)
    data['price'] = data['url'].apply(h_find_price)
    data['old_price'] = data['url'].apply(h_find_old_price)
    data['url'] = data['url'].apply(h_find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной `all_url` и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, наличия товара, названия товара, цены товара, старой цены для акционного товара.

# Парсинг цен кодов товара и url

In [46]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

переменная со списком ссылок на сайт для парсинга

In [47]:
dfh = h_parsing_2(all_url)

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [48]:
dfh = dfh.drop_duplicates().reset_index(drop=True)

удаление дублекатов

dfh.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

dfh = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [49]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-07-03,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-07-03,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-07-03,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-07-03,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-07-03,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 8396 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6271
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       74.7
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8396 entries, 0 to 8395
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           8396 non-null   object        
 1   id            8396 non-null   object        
 2   date          8396 non-null   datetime64[ns]
 3   availability  8396 non-null   object        
 4   title         8396 non-null   object        
 5   price         8396 non-null   float64       
 6   old_price     2125 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 459.3+ KB


None

In [50]:
hai_group = pd.read_csv('csv/ahay_group.csv')

загрузка таблицы с группами товара

In [51]:
hai_group = hai_group.drop_duplicates(keep=False )

In [52]:
len(hai_group)

8974

In [53]:
dfh = dfh.merge(hai_group, how='left', on='id')

обединение таблиц предварительного парсинга и таблицы групп товаров

In [54]:
dfh

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-07-03,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-07-03,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-07-03,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-07-03,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-07-03,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
...,...,...,...,...,...,...,...,...,...,...,...
8391,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-transparent-led15w-e27-5628-tkl/,Т-000085036,2024-07-03,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628) (TKL),3279.0,4099.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
8392,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-amber-led15w-e27-5625-tkl/,Т-000085037,2024-07-03,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),3311.0,4139.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
8393,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-3pl-universo-black-led15w-e27-4483-tkl/,Т-000085021,2024-07-03,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),3359.0,4199.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
8394,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-6pl-fairy-black-6w-g9-4856-tkl/,Т-000085033,2024-07-03,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3471.0,4339.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры


In [55]:
h_df_new = dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())]

создание тыблицы с товарами, у которых нет данных о группах

In [56]:
len(h_df_new)

55

In [57]:
dfh = dfh[~((dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna()))]

отчистка таблицы парсинга от пропусков

In [58]:
len(dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())])

0

dfh = dfh.sample(30)

тестовое уменьшение таблици

In [59]:
def h_xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
            
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()      
        elif len_name <= 3:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name == 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name > 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[4]).replace(',','').replace(' ','_').replace('-','_').lower()
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_14), (d_15), (d_16), (d_17)])

Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара

# Парсинг групп товара

In [60]:
if len(h_df_new) > 0:
    tqdm.pandas() 
    h_df_new[['_category','_subcategory','_group','_subgroup']] = h_df_new['url'].progress_apply(h_xxx)
else:
    pass

  0%|          | 0/55 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_4808\1426536857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_df_new[['_category','_subcategory','_group','_subgroup']] = h_df_new['url'].progress_apply(h_xxx)


Основной парсинг

In [61]:
h_data = h_df_new[~((h_df_new['_category']=='err1') | (h_df_new['_category']=='err0'))].reset_index(drop=True)
h_data = h_data[['id', '_category','_subcategory','_group', '_subgroup']]
h_data = h_data.sort_values(['_category','_subcategory', '_group', '_subgroup'])

In [62]:
len(h_data)

49

создание таблици с группами нового товара

In [63]:
hai_group = pd.concat([hai_group, h_data], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)
hai_group.to_csv('csv/ahay_group.csv', index=False)  

добавление в таблицу с группами товара новых данных

In [64]:
dfh = pd.concat([dfh, h_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)

объединение таблици со старыми и новыми товарами

In [65]:
dfh._category.unique()

array(['H_акция', 'H_бытовая_техника', 'H_компьютерная_техника',
       'H_мебель_текстиль', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_товар_без_акции',
       'H_товар_по_акции', 'H_товары_для_дома', 'err1'], dtype=object)

In [66]:
dfh._subcategory.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_apple', 'H_аксессуары',
       'H_видеотехника', 'H_готовые_решения', 'H_игровые_приставки',
       'H_комплектующие', 'H_крепления', 'H_мониторы', 'H_мультимедиа',
       'H_накопители', 'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_умный_дом_с_яндекс',
       'H_электропитание', 'H_мебель_для_офиса', 'H_техника_для_дома',
       'H_электроинструменты', 'H_аудиосистемы',
       'H_батарейки_и_зарядны

In [67]:
dfh._group.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_мульти_сплит', 'H_обогреватели',
       'H_отопительные_печи', 'H_увлажнители_комплексы',
       'H_варочные_поверхности', 'H_вытяжки', 'H_духовые_шкафы',
       'H_кронштейны_для_микроволновых_печей', 'H_микроволновые_печи',
       'H_морозильники_и_лари', 'H_плиты_газовые_и_электрические',
       'H_посудомоечные_машины', 'H_холодильники',
       'H_электропечи_и_духовки', 'H_блендеры', 'H_вакууматоры',
       'H_весы_кухонные', 'H_генераторы_льда', 'H_кухонные_комбайны',
       'H_ломтерезки', 'H_миксеры', 'H_соковыжималки',
       'H_соковыжималки_для_цитрусов', 'H_тостеры', 'H_чопперы',
       'H_электромясорубки', 'H_минимойки', 'H_кофе', 'H_кофеварки',
       'H_кофемашины', 'H_кофемолки', 'H_термопоты',
       'H_турки_эле

просмотр категорий, подкатегорий и групп товаров

In [68]:
dfh[dfh['_category']=='err1'].id.count()

6

In [69]:
dfh[dfh['_category']=='err0'].id.count()

0

In [70]:
dfh[dfh['_category']=='err1']

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
8390,https://hi-tech.md/televizory-i-elektronika/smartfony-i-aksessuary/aksessuary/chehol-dlya-smartfona-becover-apple-iphone-11-704361-prozrachnyy/,Т-000093685,2024-07-03,В наличии,Чехол для смартфона BeCover Apple iPhone 11 (704361) прозрачный,99.0,NaN,err1,err1,err1,err1
8391,https://hi-tech.md/televizory-i-elektronika/smartfony-i-aksessuary/aksessuary/chehol-dlya-smartfona-becover-apple-iphone-13-707147-prozrachnyy/,Т-000093687,2024-07-03,В наличии,Чехол для смартфона BeCover Apple iPhone 13 (707147) прозрачный,99.0,NaN,err1,err1,err1,err1
8392,https://hi-tech.md/bytovaya-tehnika/melkaya-tehnika-dlya-kuhni/choppery/chopper-saturn-st-fp0062/,Т-000067213,2024-07-03,Предзаказ,чоппер Saturn ST-FP0062,499.0,NaN,err1,err1,err1,err1
8393,https://hi-tech.md/bytovaya-tehnika/krupnaya-tehnika-dlya-kuhni/elektro-pechi-i-duhovki/elektropech-laretti-lr-ec3910/,Т-000093796,2024-07-03,Предзаказ,Электропечь Laretti LR-EC3910,2899.0,NaN,err1,err1,err1,err1
8394,https://hi-tech.md/kompyuternaya-tehnika/aksessuary/dlya-planshetov/sumki-i-chehly/dlya-planshetov/chehol-dlya-plansheta-becover-premium-for-tab-a9-sm-x115-710113-deep-blue/,Т-000093692,2024-07-03,В наличии,Чехол для планшета BeCover Premium for Tab A9 SM-X115 (710113) Deep Blue,369.0,NaN,err1,err1,err1,err1
8395,https://hi-tech.md/kompyuternaya-tehnika/aksessuary/dlya-planshetov/sumki-i-chehly/dlya-planshetov/chehol-dlya-plansheta-becover-smart-case-for-apple-ipad-10.2-2019-2020-2021-704132-black/,Т-000093682,2024-07-03,В наличии,Чехол для планшета BeCover Smart Case for Apple iPad 10.2 2019/2020/2021 (704132) Black,399.0,NaN,err1,err1,err1,err1


In [71]:
dfh[dfh['_category']=='err0']

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup


просмотр товаров с ошибками

In [72]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,2024-07-03,В наличии,кондиционер Electrolux EACS-07HAT/N3_21Y,3999.0,NaN,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-07-03,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,2024-07-03,В наличии,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,2024-07-03,В наличии,насадка д/Зубная щетка Braun EB18 3D White (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,2024-07-03,В наличии,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),94.0,NaN,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток


****************************************************************************************************
таблица имеет 8396 сторок и 11 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6271
_category          0
_subcategory       0
_group             0
_subgroup          0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       74.7
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
_category               object
_subcategory            object
_group                  object
_subgroup               object
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8396 entries, 0 to 8395
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           8396 non-null   object        
 1   id            8396 non-null   object        
 2   date          8396 non-null   datetime64[ns]
 3   availability  8396 non-null   object        
 4   title         8396 non-null   object        
 5   price         8396 non-null   float64       
 6   old_price     2125 non-null   float64       
 7   _category     8396 non-null   object        
 8   _subcategory  8396 non-null   object        
 9   _group        8396 non-null   object        
 1

None

просмотр данных о таблице

dfh.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

dfh = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

In [73]:
dfh.head(2)

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,2024-07-03,В наличии,кондиционер Electrolux EACS-07HAT/N3_21Y,3999.0,NaN,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-07-03,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток


In [74]:
dfh['error_group'] = 0
dfh.loc[dfh['_subgroup'].isna(), 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err0', 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err1', 'error_group'] = 1

Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [75]:
dfh.loc[dfh['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'H_no_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

Изменение типов данных столбцов наличия товара в определенном магазине на тип данных `float`  и создания столбцов по наличию товара в магазинах городов

In [76]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'error_group', ]]

изменение порядка располажения столбцов

In [77]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-07-03,3999.0,NaN,В наличии,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-07-03,34.0,68.0,В наличии,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-07-03,94.0,NaN,В наличии,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-07-03,94.0,NaN,В наличии,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),2024-07-03,94.0,NaN,В наличии,0


****************************************************************************************************
таблица имеет 8396 сторок и 12 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
_category          0
_subcategory       0
_group             0
_subgroup          0
title              0
date               0
price              0
old_price       6271
availability       0
error_group        0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
title            0.0
date             0.0
price            0.0
old_price       74.7
availability     0.0
error_group      0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
_category               object
_subcategory            object
_group                  object
_subgroup               object
title                   object
date            datetime64[ns]
price                  float64
old_price              float64
availability            object
error_group              int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8396 entries, 0 to 8395
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           8396 non-null   object        
 1   id            8396 non-null   object        
 2   _category     8396 non-null   object        
 3   _subcategory  8396 non-null   object        
 4   _group        8396 non-null   object        
 5   _subgroup     8396 non-null   object        
 6   title         8396 non-null   object        
 7   date          8396 non-null   datetime64[ns]
 8   price         8396 non-null   float64       
 9   old_price     2125 non-null   float64       
 1

None

просмотр данных после парсинга

# Группировка товара

- разобраться с креслами в ногруп

In [78]:
dfh.loc[dfh['_category']=='H_товар_без_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_category']=='H_товар_по_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_subcategory']=='H_мультимедиа', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_акустические_системы', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_музыкальная_техника', '_group'] = 'H_акустика'
dfh.loc[dfh['_subcategory']=='H_умный_дом_с_яндекс','_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_яндекс_станции', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_airpods', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_airpods', '_group'] = 'H_наушники'
dfh.loc[dfh['_group']=='H_apple_watch', '_group'] = 'H_часы_и_браслеты'
dfh.loc[dfh['_group']=='H_iphone', '_group'] = 'H_смартфоны'
dfh.loc[dfh['_group']=='H_ipad', '_group'] = 'H_планшеты'
dfh.loc[dfh['_group']=='H_macbook', '_group'] = 'H_ноутбуки'
dfh.loc[dfh['_group']=='H_imac', '_group'] = 'H_моноблоки'
dfh.loc[dfh['_group']=='H_часы_и_браслеты', '_subcategory'] = 'H_гаджеты'
dfh.loc[dfh['_group']=='H_смартфоны', '_subcategory'] = 'H_смартфоны_и_аксессуары'
dfh.loc[dfh['_group']=='H_планшеты', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_ноутбуки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_моноблоки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_subcategory']=='H_стиральные_и_сушильные_машины', '_group'] = 'H_стиральные_машины'
dfh.loc[dfh['_subgroup']=='H_сушильные_машины', '_group'] = 'H_сушильные_машины'
dfh.loc[dfh['_subgroup']=='H_наклейки', '_group'] = 'H_наклейки'
dfh.loc[dfh['_subgroup']=='H_подставки', '_group'] = 'H_подставки'
dfh.loc[dfh['_subgroup']=='H_замки', '_group'] = 'H_замки'
dfh.loc[dfh['_subgroup']=='H_usb_hub', '_group'] = 'H_usb_hub'
dfh.loc[dfh['_subgroup']=='H_сумки_и_чехлы', '_group'] = 'H_сумки_и_чехлы'
dfh.loc[dfh['_subgroup']=='H_защитные_стекла', '_group'] = 'H_защитные_стекла'
dfh.loc[dfh['_subcategory']=='H_мониторы','_group'] = 'H_мониторы'

In [79]:
for i in dfh['_category'].unique():
    t = dfh[dfh['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

H_акция
['H_рассрочка_12_месяцев___без_переплаты!']
----------------------------------------------------------------------------------------------------
H_бытовая_техника
['H_аксессуары_для_мелкой_бытовой_техники' 'H_климатическая_техника'
 'H_крупная_техника_для_кухни' 'H_мелкая_техника_для_кухни' 'H_минимойки'
 'H_приготовление_напитков' 'H_приготовление_пищи'
 'H_пылесосы_и_аксессуары' 'H_стиральные_и_сушильные_машины'
 'H_техника_karcher' 'H_техника_для_здоровья' 'H_техника_для_красоты'
 'H_товары_для_ухода_за_одеждой' 'H_установка_бытовой_техники']
----------------------------------------------------------------------------------------------------
H_компьютерная_техника
['H_аудиосистемы' 'H_гаджеты' 'H_готовые_решения'
 'H_смартфоны_и_аксессуары' 'H_apple' 'H_аксессуары' 'H_видеотехника'
 'H_игровые_приставки' 'H_комплектующие' 'H_крепления' 'H_мониторы'
 'H_накопители' 'H_оргтехника' 'H_периферийные_устройства'
 'H_проекторы_и_экраны' 'H_расходные_материалы' 'H_сетевое_оборудован

In [80]:
for i in dfh['_subcategory'].unique():
    t = dfh[dfh['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

H_рассрочка_12_месяцев___без_переплаты!
['H_рассрочка_12_месяцев___без_переплаты!']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_мелкой_бытовой_техники
['H_аксессуары_для_зубных_щеток' 'H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_климатическая_техника
['H_бойлеры' 'H_вентиляторы' 'H_комплектующие_для_климатической_техники'
 'H_кондиционеры' 'H_кондиционеры_мульти_сплит' 'H_обогреватели'
 'H_отопительные_печи' 'H_увлажнители_комплексы']
----------------------------------------------------------------------------------------------------
H_крупная_техника_для_кухни
['H_варочные_поверхности' 'H_вытяжки' 'H_духовые_шкафы'
 'H_кронштейны_для_микроволновых_печей' 'H_микроволновые_печи'
 'H_морозильники_и_лари' 'H_плиты_газовые_и_электрические'
 'H_посудомоечные_машины' 'H_холодильники' 'H_электропечи_и_духовки']
------------

просмотр групп товаров в подкатегориях

In [81]:
for i in dfh['_group'].unique():
    t = dfh[dfh['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

H_рассрочка_12_месяцев___без_переплаты!
['H_рассрочка_12_месяцев___без_переплаты!']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_зубных_щеток
['H_аксессуары_для_зубных_щеток']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_электрических_бритв
['H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_бойлеры
['H_бойлеры']
----------------------------------------------------------------------------------------------------
H_вентиляторы
['H_вентиляторы']
----------------------------------------------------------------------------------------------------
H_комплектующие_для_климатической_техники
['H_комплектующие_для_климатической_техники']
----------------------------------------------------------------------------------------------------
H_кондиционеры
['H_конди

H_видеорегистраторы
['H_видеорегистраторы']
----------------------------------------------------------------------------------------------------
H_неттопы
['H_неттопы']
----------------------------------------------------------------------------------------------------
H_системные_блоки
['H_системные_блоки']
----------------------------------------------------------------------------------------------------
H_электронные_книги
['H_электронные_книги']
----------------------------------------------------------------------------------------------------
H_геймпады
['H_геймпады']
----------------------------------------------------------------------------------------------------
H_приставки
['H_приставки']
----------------------------------------------------------------------------------------------------
H_блоки_питания
['H_блоки_питания']
----------------------------------------------------------------------------------------------------
H_видеокарты
['H_nvidia']
-------------------------

['H_аудиосистемы']
----------------------------------------------------------------------------------------------------
H_плееры
['H_плееры']
----------------------------------------------------------------------------------------------------
H_саундбары
['H_саундбары']
----------------------------------------------------------------------------------------------------
H_батарейки_и_зарядные_устройства
['H_батарейки_и_зарядные_устройства']
----------------------------------------------------------------------------------------------------
H_умный_дом
['H_умный_дом']
----------------------------------------------------------------------------------------------------
H_крепления_для_тв
['H_крепления_для_тв']
----------------------------------------------------------------------------------------------------
H_приставки_для_тв
['H_приставки_для_тв']
----------------------------------------------------------------------------------------------------
H_смартфоны_и_аксессуары
['H_смартфоны_и

In [82]:
dfh._category.unique()

array(['H_акция', 'H_бытовая_техника', 'H_компьютерная_техника',
       'H_мебель_текстиль', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_no_group', 'H_товары_для_дома'],
      dtype=object)

In [83]:
dfh._subcategory.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_аудиосистемы', 'H_гаджеты',
       'H_готовые_решения', 'H_смартфоны_и_аксессуары', 'H_apple',
       'H_аксессуары', 'H_видеотехника', 'H_игровые_приставки',
       'H_комплектующие', 'H_крепления', 'H_мониторы', 'H_накопители',
       'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_электропитание', 'H_мебель_для_офиса',
       'H_техника_для_дома', 'H_электроинструменты',
       'H_батарейки_и_зарядны

In [84]:
dfh._group.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_мульти_сплит', 'H_обогреватели',
       'H_отопительные_печи', 'H_увлажнители_комплексы',
       'H_варочные_поверхности', 'H_вытяжки', 'H_духовые_шкафы',
       'H_кронштейны_для_микроволновых_печей', 'H_микроволновые_печи',
       'H_морозильники_и_лари', 'H_плиты_газовые_и_электрические',
       'H_посудомоечные_машины', 'H_холодильники',
       'H_электропечи_и_духовки', 'H_блендеры', 'H_вакууматоры',
       'H_весы_кухонные', 'H_генераторы_льда', 'H_кухонные_комбайны',
       'H_ломтерезки', 'H_миксеры', 'H_соковыжималки',
       'H_соковыжималки_для_цитрусов', 'H_тостеры', 'H_чопперы',
       'H_электромясорубки', 'H_минимойки', 'H_кофе', 'H_кофеварки',
       'H_кофемашины', 'H_кофемолки', 'H_термопоты',
       'H_турки_эле

In [85]:
dfh._subgroup.unique()

array(['H_рассрочка_12_месяцев___без_переплаты!',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_мульти_сплит', 'H_обогреватели',
       'H_отопительные_печи', 'H_увлажнители_комплексы',
       'H_варочные_поверхности', 'H_вытяжки', 'H_духовые_шкафы',
       'H_кронштейны_для_микроволновых_печей', 'H_микроволновые_печи',
       'H_морозильники', 'H_морозильные_лари', 'H_газовые_плиты',
       'H_комбинированные_плиты', 'H_плиты_газовые_и_электрические',
       'H_электрические_плиты', 'H_посудомоечные_машины',
       'H_винные_шкафы', 'H_однодверные_холодильники',
       'H_с_верхней_морозильной_камерой',
       'H_с_нижней_морозильной_камерой', 'H_электропечи_и_духовки',
       'H_блендеры_погружные', 'H_блендеры_стационарные', 'H_вакууматоры',
       'H_весы_кухонные', 'H_генераторы_льда', 'H_кухонные_комбайны',
       'H_л

# Подготовка анализа

In [86]:
dfh.head(2)

,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-07-03,3999.0,NaN,В наличии,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,H_аксессуары_для_зубных_щеток,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-07-03,34.0,68.0,В наличии,0


def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

In [87]:
tqdm.pandas()
dfh['brend'] = dfh['title'].progress_apply(brends)

  0%|          | 0/8396 [00:00<?, ?it/s]

создаем столбец с брендами

In [88]:
dfh['action_sale'] = 0
dfh.loc[dfh['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [89]:
dfh['sale'] = (100 - ((dfh['price']*100)/dfh['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [90]:
def h_count_group(x):
    return dfh[dfh['_group']==x]['id'].count()    
tqdm.pandas()
dfh['count_group'] = dfh['_group'].progress_apply(h_count_group)

  0%|          | 0/8396 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [91]:
def h_count_group_presence(x):
    return dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['id'].count()   
tqdm.pandas()
dfh['activ_count_group'] = dfh['_group'].progress_apply(h_count_group_presence)

  0%|          | 0/8396 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [92]:
def h_dif_price(x):    
    max_price = dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['price'].max()
    min_price = dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
dfh['dif_price'] = dfh['_group'].progress_apply(h_dif_price)

  0%|          | 0/8396 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [93]:
dfh['group_prise'] = dfh.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [94]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group','error_group']]

In [95]:
dfh.drop('_subgroup', axis= 1 , inplace= True )

In [96]:
dfh['tir'] = 0

# объединение

In [97]:
df = pd.concat([dfh, dft], ignore_index=True)

In [98]:
df = df.drop_duplicates().reset_index(drop=True)

In [99]:
df.to_csv('csv/hay_tir_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [100]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery/kondicioner-electrolux-eacs-07hat-n321y/,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,electrolux,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-07-03,3999.0,NaN,0,NaN,стандарт,1.000000,В наличии,1,1,0,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,panasonic,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-07-03,34.0,68.0,1,50.0,бюджет,2.764706,В наличии,7,7,0,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-07-03,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-07-03,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),2024-07-03,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0


****************************************************************************************************
таблица имеет 20038 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    2
old_price            16390
action_sale              0
sale                 16390
group_prise              2
dif_price               25
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            81.8
action_sale           0.0
sale                 81.8
group_prise           0.0
dif_price             0.1
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20038 entries, 0 to 20037
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                20038 non-null  object        
 1   id                 20038 non-null  object        
 2   _category          20038 non-null  object        
 3   _subcategory       20038 non-null  object        
 4   _group             20038 non-null  object        
 5   brend              20038 non-null  object        
 6   title              20038 non-null  object        
 7   date               20038 non-null  datetime64[ns]
 8   price              20036 non-null  float64   

None